### Dog breed classifier

In [ ]:
import numpy as np 
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import cv2
import gc

In [ ]:
# Create a list with the filepaths for training and testing
train_img_Path = '../input/dog-breed-identification/train'

test_img_Path = '../input/dog-breed-identification/test'

labels = pd.read_csv(r'../input/dog-breed-identification/labels.csv')

sample_submission = pd.read_csv(r'../input/dog-breed-identification/sample_submission.csv')

In [ ]:
labels.head()

* this dataset have 120 dogs breeds but in the given task predict only on 10 breeds.

In [ ]:
print(f'Number of pictures in the training dataset: {labels.shape[0]}\n')
print(f'Number of different labels: {len(labels.breed.unique())}\n')
print(f'Labels: {labels.breed.unique()}')

In [ ]:
labels['breed'].value_counts()

In [ ]:
labels['id'] = labels['id'] + '.jpg'

In [ ]:
plt.figure(figsize=(20,40))
i=1
for idx,s in labels.head(6).iterrows():
    img_path = os.path.join(train_img_Path,s['id'])
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    fig=plt.subplot(6,2,i)
    fig.imshow(img)
    fig.set_title(s['breed'])
    i+=1

In [ ]:
#Extracting different classes
dog_breeds = sorted(labels['breed'].unique())
n_classes = len(dog_breeds)
print(n_classes)

In [ ]:
#Converting classes to numbers
class_to_num = dict(zip(dog_breeds,range(n_classes)))

In [ ]:
#Function to load and convert images to array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical

def images_to_array(data_dir,df,image_size):
    image_names = df['id']
    image_labels = df['breed']
    data_size = len(image_names)
    
    X = np.zeros([data_size,image_size[0],image_size[1],image_size[2]],dtype = np.uint8)
    y = np.zeros([data_size,1],dtype = np.uint8)
    
    for i in range(data_size):
        img_name = image_names[i]
        img_dir = os.path.join(data_dir,img_name)
        img_pixels = load_img(img_dir,target_size=image_size)
        X[i] = img_pixels
        y[i] = class_to_num[image_labels[i]]
        
    y = to_categorical(y)
    ind = np.random.permutation(data_size)
    X = X[ind]
    y = y[ind]
    print('Ouptut Data Size: ', X.shape)
    print('Ouptut Label Size: ', y.shape)
    return X, y  

In [ ]:
#Selecting image size according to pretrained models
img_size = (299,299,3)
X, y = images_to_array(train_img_Path,labels,img_size)

In [ ]:
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D,Lambda, Dropout, InputLayer, Input

def get_features(model_name, data_preprocessor,weight, input_size, data):
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)
    
    base_model = model_name(weights=weight,
                            include_top=False,
                            input_shape=input_size)(preprocessor)
    
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    
    #Extract feature.
    feature_maps = feature_extractor.predict(data, batch_size=32, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps


In [ ]:
#Extracting features using Xception
from tensorflow import keras
from keras.applications import Xception
from keras.applications.xception import preprocess_input
Xception_preprocessor = preprocess_input
Xception_features = get_features(Xception,
                                  Xception_preprocessor,
                                 '../input/d/aeryss/keras-pretrained-models/Xception_NoTop_ImageNet.h5',
                                  img_size, X)

In [ ]:
#Extracting features using NASNetMobile
from keras.applications import NASNetMobile
from keras.applications.nasnet import preprocess_input
NASNetMobile_preprocessor = preprocess_input
NASNetMobile_features = get_features(NASNetMobile,
                                  NASNetMobile_preprocessor,
                                 '../input/d/aeryss/keras-pretrained-models/NASNetMobile_NoTop_ImageNet.h5',
                                  img_size, X)

In [ ]:
#Extracting features using InceptionResNetV2
from keras.applications import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
InceptionResNetV2_preprocessor = preprocess_input
InceptionResNetV2_features = get_features(InceptionResNetV2,
                                  InceptionResNetV2_preprocessor,
                                 '../input/d/aeryss/keras-pretrained-models/InceptionResNetV2_NoTop_ImageNet.h5',
                                  img_size, X)

In [ ]:
del X #to free up some ram memory
gc.collect()

In [ ]:
#Creating final featuremap by combining all extracted features

final_features = np.concatenate([Xception_features,
                                 NASNetMobile_features,
                                 InceptionResNetV2_features,], axis=-1) #axis=-1 to concatinate horizontally

print('Final feature maps shape', final_features.shape)

In [ ]:
#Callbacks
from keras.callbacks import EarlyStopping
EarlyStop_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
my_callback=[EarlyStop_callback]

In [ ]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
#Building Model
from keras.models import Sequential
model = Sequential()
model.add(InputLayer(final_features.shape[1:]))
model.add(Dropout(0.7))
model.add(Dense(120,activation='softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

# Training the CNN on the Train features and evaluating it on the val data
history = model.fit(final_features,y,validation_split=0.1,callbacks=my_callback, epochs = 50, batch_size=32)

In [ ]:
#deleting to free up ram memory

del NASNetMobile_features
del Xception_features
del InceptionResNetV2_features
del final_features
gc.collect()

In [ ]:
#Converting test images to array
def images_to_array2(data_dir,df, img_size):
    images_names = df['id']
    data_size = len(images_names)
    X = np.zeros([data_size, img_size[0], img_size[1], 3], dtype=np.uint8)
    
    for i in range(data_size):
        image_name = images_names[i]
        img_dir = os.path.join(data_dir, image_name+'.jpg')
        img_pixels = load_img(img_dir, target_size=img_size)
        X[i] = img_pixels
        
    print('Ouptut Data Size: ', X.shape)
    return X

In [ ]:
test_data = images_to_array2(test_img_Path, sample_submission, img_size)

In [ ]:
#Extract test data features.
def extact_features(data):
    
    Xception_features = get_features(Xception,
                                     Xception_preprocessor,
                                     '../input/d/aeryss/keras-pretrained-models/Xception_NoTop_ImageNet.h5',
                                     img_size,
                                     data)
    
    NASNetMobile_features = get_features(NASNetMobile,
                                         NASNetMobile_preprocessor,
                                         '../input/d/aeryss/keras-pretrained-models/NASNetMobile_NoTop_ImageNet.h5',
                                         img_size, 
                                         data)
    
    InceptionResNetV2_features = get_features(InceptionResNetV2,
                                              InceptionResNetV2_preprocessor,
                                              '../input/d/aeryss/keras-pretrained-models/InceptionResNetV2_NoTop_ImageNet.h5',
                                              img_size, 
                                              data)

    final_features = np.concatenate([Xception_features,
                                 NASNetMobile_features,
                                 InceptionResNetV2_features,], axis=-1)
    
    print('Final feature maps shape', final_features.shape)
    #deleting to free up ram memory
    
    del Xception_features
    del NASNetMobile_features
    del InceptionResNetV2_features
    gc.collect()
    
    
    return final_features

In [ ]:
test_features = extact_features(test_data)

In [ ]:
y_pred = model.predict(test_features, batch_size=32)

In [ ]:
for breed in dog_breeds:
    sample_submission[breed] = y_pred[:,class_to_num[breed]]
sample_submission.to_csv('pred.csv', index=None)
sample_submission

In [ ]:
# saving model for further use
model.save('dogs_breed.h5')